# Target Propagation - Tutorial 1
## Standard Target Propagation

A simple implementation of target propagation to confirm that it works.

In [ ]:
import initialize

In [ ]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms

from tools.modules import Sign, Stochastic, Clamp
from tools.training import train, classify
from tools import training
from tools.learners.target_prop import TargetPropLearner, AlternateTraining
from tools.learners.target_prop import BaselineLearner1
from functools import partial
from torch import nn

# Steps

1) Create each layer (AutoencoderLearner)
2) Create the TargetPropLearner
3) Run the training on the baseline
4) Run the training on the target propagation learner 

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

training_dataset = FashionMNIST(
    '../../Datasets/',
    transform=transform, download=True
)

testing_dataset = FashionMNIST(
    '../../Datasets/', train=False,
    transform=transform, download=True
)

baseline_loss= {}
baseline_class= {}
tp_loss = {}
tp_class = {}
tp_alt_loss = {}
tp_alt_class = {}


# Baseline Learner

Train baseline learners on 
 - LeakyReLU 
 - Sign
 - Stochastic

Use straight-through-estimators for the latter two

In [ ]:
act = 'stochastic'


# if act == 'leaky_relu':
# elif act == 'sign':
#     activation = partial(Sign, True)
# elif act == 'stochastic':
#     activation = partial(Stochastic, True, True)

activation = nn.LeakyReLU

learner = BaselineLearner1(
    784, 300, 300, 300, 10, activation=activation
)
baseline_loss[act] = train(learner, training_dataset, 20, device='cpu')
baseline_class[act] = classify(learner, testing_dataset)

# TargetPropLearner

In [ ]:
from torch import nn

act = 'leaky_relu'

for act in ['leaky_relu', 'sign', 'stochastic']:
    print('Activation: ', act)
    if act == 'leaky_relu':
        activation = nn.LeakyReLU
        in_act = None
    elif act == 'sign':
        activation = nn.Tanh
        in_act = partial(Sign, False)
    elif act == 'stochastic':
        in_act = partial(Stochastic, False, False)
        activation = nn.Sigmoid

    learner = TargetPropLearner(
        784, 300, 300, 300, 10, dropout_p=0.1, act=activation, out_x_lr=1e-3
    )
    alternator = AlternateTraining(learner, 1, 1)

    tp_loss[act] = train(learner, training_dataset, 20, device='cpu', callback=None)
    tp_class[act] = classify(learner, testing_dataset)

In [ ]:
from torch import nn

act = 'leaky_relu'

for act in ['leaky_relu', 'sign', 'stochastic']:

    if act == 'leaky_relu':
        activation = nn.LeakyReLU
        in_act = None
    elif act == 'sign':
        activation = nn.Tanh
        in_act = partial(Sign, False)
    elif act == 'stochastic':
        in_act = partial(Stochastic, False, False)
        activation = nn.Sigmoid

    learner = TargetPropLearner(
        784, 300, 300, 300, 10, act=activation, reverse_act=activation, in_act=in_act, out_x_lr=1., use_norm=True
    )

    alternator = AlternateTraining(learner, 1, 4)

    tp_alt_loss[act] = train(learner, training_dataset, 20, device='cpu', callback=alternator)
    tp_alt_class[act] = classify(learner, testing_dataset)

In [ ]:
print(baseline_loss.keys())

In [ ]:
training.plot_loss_line(
    [baseline_loss['stochastic'], tp_loss['leaky_relu'], tp_loss['sign'], tp_loss['stochastic']], 
    ['Baseline', 'Target Prop - Leaky ReLU', 'Target Prop - Sign', 'Target Prop - Stochastic'], 
    'Training Loss', save_file='images/t2x1_target_prop_2024_10_3_1.png'
)

In [ ]:
print(
    baseline_class['stochastic'], tp_class['leaky_relu'], 
    tp_class['sign'], tp_class['stochastic']
)

In [ ]:
import pickle

losses = {'target1': target_loss, 'target2': target_loss2}

with open('results/t2x1_loss_results1.pkl', 'wb') as file:
    pickle.dump(losses, file)